In [ ]:
%autosave 0

# 8. Neural networks and deep learning  
  
learn about neural nets and build a model for classifying images of clothes  

# 8.1 Fashion classification  
  
Dataset:  
  
* Full: https://github.com/alexeygrigorev/clothing-dataset
* Small: https://github.com/alexeygrigorev/clothing-dataset-small



In [ ]:
# use http to avoid ssh credential errors in Colab
#!git clone git@github.com:alexeygrigorev/clothing-dataset-small.git
!git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

In [ ]:
!ls ./clothing-dataset-small/train/t-shirt/ | head

## 8.2 TensorFlow and Keras  
  
* Installing TensorFlow  
* Loading images  
  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
path = './clothing-dataset-small/train/t-shirt'
#name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
name = '00003aeb-ace5-43bf-9a0c-dc31a03e9cd2.jpg'
fullname = f'{path}/{name}'
load_img(fullname)

In [ ]:
# can resize to fit expectation of model
img = load_img(fullname, target_size=(299, 299))

In [ ]:
x = np.array(img)
x.shape


## 8.3 Pre-trained convolutional neural networks  
  
* Imagenet dataset: https://www.image-net.org/
* Pre-trained models: https://keras.io/api/applications/


In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [ ]:
model = Xception(weights='imagenet', input_shape=(299, 299, 3))

In [ ]:
# convert to array
X = np.array([x])

# if multiple images {x, y, z} would be:
#X = np.array([x, y, z])


In [ ]:
X.shape

In [ ]:
#  apply necessary model pre-processing to get valid predictions
X = preprocess_input(X)

In [ ]:
pred = model.predict(X)

In [ ]:
# predictions for top classes
decode_predictions(pred)

# 8.4 Convolutional neural networks  
  
 * Types of layers: convolutional and dense
 * Convolutional layers and filters
 * Dense layers


# 8.5 Transfer Learning

In [ ]:
# note: deprecacted. consider using `tensorflow.keras.utils.image_dataset_from_directory`
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(150, 150),
    batch_size=32
)


In [ ]:
# class names inferred from named directories
!ls -l clothing-dataset-small/train

In [ ]:
train_ds.class_indices

In [ ]:
X, y = next(train_ds)

In [ ]:
# {batch size, image width, image height, color channels}
X.shape

In [ ]:
# {batch size, multi-class size}
y.shape

In [ ]:
y[:5]

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

In [ ]:
# Retrain new model
#  -- drop the 'Dense Layers' `include_top=False`
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)
# Keep 'Convolutional Layers' static during retraining
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, input_size, 3))

base = base_model(inputs)

outputs = base

model = keras.Model(inputs, outputs)

In [ ]:
# {batch size, 3D outcome field}
preds.shape

In [ ]:

inputs = keras.Input(shape=(input_size, input_size, 3))

base = base_model(inputs, training=False)
# Pooling: collapse 3D outcome prediction to vector
#pooling = keras.layers.GlobalAveragePooling2D()
#vectors = pooling(base)
vectors = keras.layers.GlobalAveragePooling2D()(base)


#outputs = vectors # prediction for all classes '2048'
outputs = keras.layers.Dense(10)(vectors) # Fix 'Dense Layer' to 10 classes

model = keras.Model(inputs, outputs)

In [ ]:
# instantiate optimizer and loss function:

learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
#plt.plot(history.history['accuracy'], label='train') # training accuracy
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()

# 8.6 Adjusting the learning rate

* What's the learning rate
* Trying different values

In [ ]:
# Adjust learning rate  
def make_model(learning_rate=0.01):
    base_model = Xception(
        weights='imagenet',
        include_top=False, # drop the 'Dense Layers' to retrain 
        input_shape=(150, 150, 3)
    )
    # Keep 'Convolutional Layers' static during retraining
    base_model.trainable = False

    #########################################

    # create architecture

    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    # Pooling: collapse 3D outcome prediction to vector
    #pooling = keras.layers.GlobalAveragePooling2D()
    #vectors = pooling(base)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    #outputs = vectors # prediction for all classes '2048'
    outputs = keras.layers.Dense(10)(vectors) # Fix 'Dense Layer' to 10 classes
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
scores = {}

#for lr in [0.0001, 0.001, 0.01, 0.1]:
for lr in [0.001, 0.01]:
    print(lr)

    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history

    print()
    print()

In [ ]:
for lr, hist in scores.items():
    #plt.plot(hist['accuracy'], label=('train=%s' % lr))
    plt.plot(hist['val_accuracy'], label=('val=%s' % lr))

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
learning_rate = 0.001

# 8.7 Checkpointing

* Saving the best model only
* Training a model with callbacks


In [ ]:
# save model name (eg. 'xception_v1_01_0.771.h5')
# formats:
# * `02d` two digits (eg. "05", "11")
# * `.3f` three decimal places (eg. "0.001", "0.100")
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True, # only save model if epoch result is better
    monitor='val_accuracy',
    mode='max' # want largest scoring defined in 'monitor'
)

In [ ]:
learning_rate = 0.001

model = make_model(learning_rate=learning_rate)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

In [ ]:
# review saved models
!ls -l 

# 8.8 Adding more layers  
  
* Adding one inner dense layer  
* Experimenting with different sizes of inner lay  

In [ ]:
def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(
        weights='imagenet',
        include_top=False, # drop the 'Dense Layers' to retrain 
        input_shape=(150, 150, 3)
    )
    # Keep 'Convolutional Layers' static during retraining
    base_model.trainable = False

    #########################################

    # create architecture

    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    # Pooling: collapse 3D outcome prediction to vector
    #pooling = keras.layers.GlobalAveragePooling2D()
    #vectors = pooling(base)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    
    #outputs = vectors # prediction for all classes '2048'
    #outputs = keras.layers.Dense(10)(vectors) # Fix 'Dense Layer' to 10 classes
    outputs = keras.layers.Dense(10)(inner)
    
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [ ]:
learning_rate = 0.001

scores = {}

for size in [10, 100, 1000]:
    print(size)

    model = make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history

    print()
    print()

# 8.9 Regularization and dropout  
  
* Regularizing by freezing a part of the network  
* Adding dropout to our model  
* Experimenting with different values  

In [ ]:
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
        weights='imagenet',
        include_top=False, # drop the 'Dense Layers' to retrain 
        input_shape=(150, 150, 3)
    )
    # Keep 'Convolutional Layers' static during retraining
    base_model.trainable = False

    #########################################

    # create architecture

    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    # Pooling: collapse 3D outcome prediction to vector
    #pooling = keras.layers.GlobalAveragePooling2D()
    #vectors = pooling(base)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)

    # dropout layer: random proportion of nodes dropped each training epoch
    drop = keras.layers.Dropout(droprate)(inner)
    
    #outputs = vectors # prediction for all classes '2048'
    #outputs = keras.layers.Dense(10)(vectors) # Fix 'Dense Layer' to 10 classes
    outputs = keras.layers.Dense(10)(inner)
    
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

# 8.10 Data augmentation  
  
* Different data augmentations
* Training a model with augmentations
* How to select data augmentations?


In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
     # adjust images to make training data more robust
     shear_range=10.0,    
     zoom_range=0.1,
     vertical_flip=True,
)

train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(150, 150),
    batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

# 8.11 Training a larger model  
  
* Train a 299x299 model  
  

In [ ]:
def make_model(
    # image size (increase accuracy with larger image, slower training)
    input_size = 150,
    learning_rate=0.01, size_inner=100):
    base_model = Xception(
        weights='imagenet',
        include_top=False, # drop the 'Dense Layers' to retrain 
        input_shape=(input_size, input_size, 3)
    )
    # Keep 'Convolutional Layers' static during retraining
    base_model.trainable = False

    #########################################

    # create architecture

    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False)
    # Pooling: collapse 3D outcome prediction to vector
    #pooling = keras.layers.GlobalAveragePooling2D()
    #vectors = pooling(base)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    
    #outputs = vectors # prediction for all classes '2048'
    #outputs = keras.layers.Dense(10)(vectors) # Fix 'Dense Layer' to 10 classes
    outputs = keras.layers.Dense(10)(inner)
    
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model


In [ ]:
# image size (eg. 150 -> 150 x 150)
# -- (increase accuracy with larger image, slower training)
input_size = 299,

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    # adjust images to make training data more robust
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(input_size, input_size),
    batch_size=32
)


val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(input_size, input_size),
    batch_size=32,
    shuffle=False
)

In [ ]:
# save model name (eg. 'xception_v1_01_0.771.h5')
# formats:
# * `02d` two digits (eg. "05", "11")
# * `.3f` three decimal places (eg. "0.001", "0.100")
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v4_1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,  # only save model if epoch result is better
    monitor='val_accuracy',
    mode='max' # want largest scoring defined in 'monitor'
)

In [ ]:
learning_rate = 0.0005
size = 100
droprate = 0.2

model = make_model(
    input_size=input_size,
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate
)

history = model.fit(train_ds, epochs=50, validation_data=val_ds,
                   callbacks=[checkpoint])

# 8.12 Using the model  
  
* Loading the model  
* Evaluating the model  
* Getting predictions  
  

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.applications.xception import preprocess_input

import numpy as np

In [ ]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = test_gen.flow_from_directory(
    './clothing-dataset-small/test',
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)

In [ ]:
model = keras.models.load_model('xception_v4_1_13_0.903.h5')

In [ ]:
model.evaluate(test_ds)

In [ ]:
path = 'clothing-dataset-small/test/pants/c8d21106-bbdb-4e8d-83e4-bf3d14e54c16.jpg'

In [ ]:
img = load_img(path, target_size=(299, 299))

In [ ]:
x = np.array(img)
X = np.array([x])


In [ ]:
# (1, 299, 299, 3)
X.shape

In [ ]:
X = preprocess_input(X)

In [ ]:
pred = model.predict(X)

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [ ]:
dict(zip(classes, pred[0]))